In [1]:
import pandas as pd
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
from scipy.stats import pearsonr
import plotly.express as px
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [182]:
egx = pd.read_csv('egxsentiment.csv').drop(columns = ['Unnamed: 0','News']).rename(columns = {'Sentiment':'EGX_SENT'})
egx.count()

Date        221
EGX_SENT    221
dtype: int64

In [183]:
inv = pd.read_csv('investmentsentiment.csv').drop(columns = ['Unnamed: 0','News']).rename(columns = {'sentiment':'INV_SENT'})
inv.count()

Date        450
INV_SENT    299
dtype: int64

In [184]:
stock = pd.read_csv('EGX 30 Historical Data final.csv').rename(columns = {'Change %':'Change'})

In [185]:
stock.head()

,Date,Price,Open,High,Low,Vol.,Change,change_percentage
0,03/26/2024,"29,057.50","31,013.88","29,532.31","28,997.13",211.16M,-0.98%,0
1,03/25/2024,"29,344.46","31,013.88","29,344.46","28,946.14",1.19M,0.98%,1
2,03/24/2024,"29,060.02","31,013.88","29,377.43","28,777.22",1.75M,0.02%,1
3,03/21/2024,"29,054.96","31,013.88","29,094.10","28,280.76",2.80M,1.63%,1
4,03/20/2024,"28,589.37","28,675.70","29,229.67","28,542.99",177.70M,-0.30%,0


In [186]:
stock['Date'] = pd.to_datetime(stock['Date'])
stock['Date'] = stock['Date'].dt.strftime('%Y-%m-%d')

egx['Date'] = pd.to_datetime(stock['Date'])
egx['Date'] = egx['Date'].dt.strftime('%Y-%m-%d')

inv['Date'] = pd.to_datetime(stock['Date']) 
inv['Date'] = inv['Date'].dt.strftime('%Y-%m-%d')
stock.head()

,Date,Price,Open,High,Low,Vol.,Change,change_percentage
0,2024-03-26,"29,057.50","31,013.88","29,532.31","28,997.13",211.16M,-0.98%,0
1,2024-03-25,"29,344.46","31,013.88","29,344.46","28,946.14",1.19M,0.98%,1
2,2024-03-24,"29,060.02","31,013.88","29,377.43","28,777.22",1.75M,0.02%,1
3,2024-03-21,"29,054.96","31,013.88","29,094.10","28,280.76",2.80M,1.63%,1
4,2024-03-20,"28,589.37","28,675.70","29,229.67","28,542.99",177.70M,-0.30%,0


In [187]:
df = stock.merge(egx, on='Date', how='left')
df = df.merge(inv, on='Date', how='left')
df.head()

,Date,Price,Open,High,Low,Vol.,Change,change_percentage,EGX_SENT,INV_SENT
0,2024-03-26,"29,057.50","31,013.88","29,532.31","28,997.13",211.16M,-0.98%,0,negative,positive
1,2024-03-25,"29,344.46","31,013.88","29,344.46","28,946.14",1.19M,0.98%,1,positive,positive
2,2024-03-24,"29,060.02","31,013.88","29,377.43","28,777.22",1.75M,0.02%,1,negative,positive
3,2024-03-21,"29,054.96","31,013.88","29,094.10","28,280.76",2.80M,1.63%,1,negative,positive
4,2024-03-20,"28,589.37","28,675.70","29,229.67","28,542.99",177.70M,-0.30%,0,positive,neutral


In [188]:
df['EGX_SENT'] = df['EGX_SENT'].fillna('neutral')
df['INV_SENT']= df['INV_SENT'].fillna('neutral')

In [189]:
df['Change'] = df['Change'].str.replace('%', '').astype(float)
df['Price'] =  df['Price'].str.replace(',','').astype(float)

df

,Date,Price,Open,High,Low,Vol.,Change,change_percentage,EGX_SENT,INV_SENT
0,2024-03-26,29057.50,"31,013.88","29,532.31","28,997.13",211.16M,-0.98,0,negative,positive
1,2024-03-25,29344.46,"31,013.88","29,344.46","28,946.14",1.19M,0.98,1,positive,positive
2,2024-03-24,29060.02,"31,013.88","29,377.43","28,777.22",1.75M,0.02,1,negative,positive
3,2024-03-21,29054.96,"31,013.88","29,094.10","28,280.76",2.80M,1.63,1,negative,positive
4,2024-03-20,28589.37,"28,675.70","29,229.67","28,542.99",177.70M,-0.30,0,positive,neutral
...,...,...,...,...,...,...,...,...,...,...
540,2022-01-09,11978.56,"12,043.42","12,044.78","11,958.61",59.66M,-0.54,0,neutral,neutral
541,2022-01-05,12043.42,"11,982.61","12,069.23","11,980.87",144.60M,0.51,1,neutral,neutral
542,2022-01-04,11982.61,"11,897.40","11,984.21","11,897.15",114.48M,0.72,1,neutral,neutral
543,2022-01-03,11897.40,"11,909.72","11,916.04","11,870.47",79.32M,-0.10,0,neutral,neutral


In [190]:
sentiment_map={'positive':1, 'neutral':0, 'negative': -1}
df['EGX_SENT'] = df['EGX_SENT'].map(sentiment_map)
df['INV_SENT'] = df['INV_SENT'].map(sentiment_map)

df

,Date,Price,Open,High,Low,Vol.,Change,change_percentage,EGX_SENT,INV_SENT
0,2024-03-26,29057.50,"31,013.88","29,532.31","28,997.13",211.16M,-0.98,0,-1,1
1,2024-03-25,29344.46,"31,013.88","29,344.46","28,946.14",1.19M,0.98,1,1,1
2,2024-03-24,29060.02,"31,013.88","29,377.43","28,777.22",1.75M,0.02,1,-1,1
3,2024-03-21,29054.96,"31,013.88","29,094.10","28,280.76",2.80M,1.63,1,-1,1
4,2024-03-20,28589.37,"28,675.70","29,229.67","28,542.99",177.70M,-0.30,0,1,0
...,...,...,...,...,...,...,...,...,...,...
540,2022-01-09,11978.56,"12,043.42","12,044.78","11,958.61",59.66M,-0.54,0,0,0
541,2022-01-05,12043.42,"11,982.61","12,069.23","11,980.87",144.60M,0.51,1,0,0
542,2022-01-04,11982.61,"11,897.40","11,984.21","11,897.15",114.48M,0.72,1,0,0
543,2022-01-03,11897.40,"11,909.72","11,916.04","11,870.47",79.32M,-0.10,0,0,0


In [191]:
df['EGX_SENT'] = df['EGX_SENT'].shift(-1)
df['INV_SENT'] = df['INV_SENT'].shift(-1)

df = df[['Date','Price','change_percentage','Change','EGX_SENT','INV_SENT']]

In [192]:
df

,Date,Price,change_percentage,Change,EGX_SENT,INV_SENT
0,2024-03-26,29057.50,0,-0.98,1.0,1.0
1,2024-03-25,29344.46,1,0.98,-1.0,1.0
2,2024-03-24,29060.02,1,0.02,-1.0,1.0
3,2024-03-21,29054.96,1,1.63,1.0,0.0
4,2024-03-20,28589.37,0,-0.30,-1.0,-1.0
...,...,...,...,...,...,...
540,2022-01-09,11978.56,0,-0.54,0.0,0.0
541,2022-01-05,12043.42,1,0.51,0.0,0.0
542,2022-01-04,11982.61,1,0.72,0.0,0.0
543,2022-01-03,11897.40,0,-0.10,0.0,0.0


In [193]:
import plotly.express as px

fig = px.line(df, x="Date", y="Change", markers=True)
fig.show()

In [194]:
df= df.sort_values(by='Date')
df.reset_index(drop=True, inplace=True)

In [195]:
train = df[df['Date']<'2023-03-01']
test = df[df['Date']>='2023-03-01']

In [196]:
train.fillna(0,inplace=True)
train

C:\Users\Farah\AppData\Local\Temp\ipykernel_11360\2246764937.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Price,change_percentage,Change,EGX_SENT,INV_SENT
0,2022-01-02,11909.72,0,-0.33,0.0,0.0
1,2022-01-03,11897.40,0,-0.10,0.0,0.0
2,2022-01-04,11982.61,1,0.72,0.0,0.0
3,2022-01-05,12043.42,1,0.51,0.0,0.0
4,2022-01-09,11978.56,0,-0.54,0.0,0.0
...,...,...,...,...,...,...
279,2023-02-22,16777.20,0,-1.54,0.0,1.0
280,2023-02-23,17003.90,1,1.35,0.0,1.0
281,2023-02-26,17213.01,1,1.23,0.0,0.0
282,2023-02-27,17098.16,0,-0.67,0.0,0.0


In [197]:
exog_vars = ['INV_SENT', 'EGX_SENT']


In [198]:
sarimax_model = SARIMAX(train['change_percentage'], exog=train[exog_vars], order=(2,2,2), seasonal_order=(0, 1, 0, 12))

sarimax_results = sarimax_model.fit()


D:\anaconda\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

D:\anaconda\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [199]:
sarimax_forecast = sarimax_results.predict(start=test.index[0], end=test.index[-1], exog=test[exog_vars])


In [200]:

test['pred'] = sarimax_forecast


C:\Users\Farah\AppData\Local\Temp\ipykernel_11360\599234636.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [201]:
import plotly.express as px
fig = px.line(train, x="Date", y="change_percentage", markers=True)
fig.add_scatter(x=test["Date"], y=test["change_percentage"], mode="lines", name="Test Data",line_color="red")
fig.add_scatter(x=test["Date"], y=test["pred"], mode="lines", name="Predicted Data", line_color="green")

fig.show()

In [202]:
import plotly.graph_objects as go


fig = go.Figure()


fig.add_scatter(x=test["Date"], y=test["change_percentage"], mode="lines", name="Test Data", line_color="red")


fig.add_scatter(x=test["Date"], y=test["pred"], mode="lines", name="Predicted Data", line_color="green")

fig.show()


In [203]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


mae = mean_absolute_error(test['change_percentage'], test['pred'])


mse = mean_squared_error(test['change_percentage'], test['pred'])


rmse = mean_squared_error(test['change_percentage'], test['pred'], squared=False)


print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)


Mean Absolute Error (MAE): 0.6232293289114569
Mean Squared Error (MSE): 0.6122522860312074
Root Mean Squared Error (RMSE): 0.7824655174710304


In [24]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
# Grid search
# Adjusting model parameters and fitting options 
p, d, q = 2, 2, 2
P, D, Q, s = 0, 1, 0, 12 

try:
    model = SARIMAX(train['Change'],
                    exog=train[exog_vars],
                    order=(p, d, q),
                    seasonal_order=(P, D, Q, s),
                    enforce_stationarity=True,
                    enforce_invertibility=False)
    results = model.fit(maxiter=200, method='nm', disp=False)  #  fitting method and iterations
    print(results.summary())
except Exception as e:
    print("Model fitting failed:", e)


if not results.mle_retvals['converged']:
    print("Model has not converged")
else:
    print("Model converged successfully")


D:\anaconda\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.



                                      SARIMAX Results                                       
Dep. Variable:                               Change   No. Observations:                  284
Model:             SARIMAX(2, 2, 2)x(0, 1, [1], 12)   Log Likelihood                -693.480
Date:                              Mon, 06 May 2024   AIC                           1402.961
Time:                                      14:14:28   BIC                           1431.748
Sample:                                           0   HQIC                          1414.520
                                              - 284                                         
Covariance Type:                                opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
INV_SENT      -0.6312      0.411     -1.536      0.124      -1.437       0.174
EGX_SENT      -0.01

D:\anaconda\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

